In [2]:
# Eclipse Viewer — Observer's Perspective
Simulates what an eclipse looks like from any location on Earth.  
Select an eclipse, set your latitude/longitude, and use the **time slider** to watch the Moon cross the Sun.

SyntaxError: invalid syntax (2612845927.py, line 2)

In [19]:
import json
import re
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, Dropdown

%matplotlib inline

In [ ]:
# ============================================================
# Load eclipse data exported by EclipseData.ipynb
# ============================================================
with open("eclipse_data.json") as f:
    data = json.load(f)
#loading in the json file that contains the parsed eclipse data in order for the visualization functions to calculate position given our data points.#
eclipse_list = data["eclipse_list"]
print(f"Loaded {len(eclipse_list)} eclipses from: {data['source_url']}")
print(f"Date range: {eclipse_list[0]['date_raw']}  →  {eclipse_list[-1]['date_raw']}")

Loaded 224 eclipses from: https://eclipse.gsfc.nasa.gov/SEcat5/SE2001-2100.html
Date range: 2001 Jun 21  →  2100 Sep 04


In [ ]:
# ============================================================
# GEOMETRY HELPERS
# ============================================================

def parse_coord(coord_str):
    """Parse NASA coordinate string like '11S' or '131W' to signed degrees."""
    if not coord_str or coord_str.strip() == "-":
        return 0.0
    match = re.match(r"(\d+)([NSEW])", coord_str.strip())
    if match:
        val = float(match.group(1))
        if match.group(2) in ("S", "W"):
            val = -val
        return val
    return 0.0


def haversine_km(lat1, lon1, lat2, lon2):
    """Great-circle distance in km between two points on Earth."""
    R = 6371.0
    la1, lo1, la2, lo2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = la2 - la1
    dlon = lo2 - lo1
    a = np.sin(dlat / 2) ** 2 + np.cos(la1) * np.cos(la2) * np.sin(dlon / 2) ** 2
    return R * 2 * np.arcsin(np.sqrt(np.clip(a, 0, 1)))


def overlap_fraction(sun_r, moon_r, d):
    """
    Fraction of the Sun's disk area obscured by the Moon.

    sun_r, moon_r : apparent radii (same units)
    d             : center-to-center distance
    """
    if d >= sun_r + moon_r:
        return 0.0
    if d <= abs(sun_r - moon_r):
        return 1.0 if moon_r >= sun_r else (moon_r / sun_r) ** 2

    # Partial overlap — lens-shaped intersection of two circles
    cos1 = np.clip((d**2 + sun_r**2 - moon_r**2) / (2 * d * sun_r), -1, 1)
    cos2 = np.clip((d**2 + moon_r**2 - sun_r**2) / (2 * d * moon_r), -1, 1)
    a1 = sun_r**2 * np.arccos(cos1)
    a2 = moon_r**2 * np.arccos(cos2)
    det = (-d + sun_r + moon_r) * (d + sun_r - moon_r) * \
          (d - sun_r + moon_r) * (d + sun_r + moon_r)
    a3 = 0.5 * np.sqrt(max(det, 0))
    return (a1 + a2 - a3) / (np.pi * sun_r**2)


MONTH_NUM = {
    "Jan": 1, "Feb": 2, "Mar": 3, "Apr": 4,
    "May": 5, "Jun": 6, "Jul": 7, "Aug": 8,
    "Sep": 9, "Oct": 10, "Nov": 11, "Dec": 12,
}


# ============================================================
# ECLIPSE CLASS
# ============================================================

class Eclipse:
    """
    Represents a single solar eclipse with methods for observer
    visibility calculations using parallax-based geometry.

    The Moon's topocentric parallax (~0.95°) shifts its apparent
    position for observers displaced from the shadow centerline.
    The known path width calibrates this angular conversion so that
    the edge of totality maps exactly to the Moon-Sun radius
    difference (magnitude − 1).
    """

    # Average Earth-Moon distance (km) — used for parallax
    MOON_DISTANCE_KM = 384_400
    # Sun angular radius (radians, ~16 arcmin)
    SUN_ANGULAR_RAD = 0.00465

    def __init__(self, eclipse_dict):
        self.data = eclipse_dict
        self._raw = eclipse_dict.get("_raw", {})

        # Core properties
        self.date_raw = eclipse_dict.get("date_raw", "")
        self.year = eclipse_dict.get("year", 0)
        self.type_code = eclipse_dict.get("type_code", "")
        self.type_name = eclipse_dict.get("type", "")
        self.magnitude = eclipse_dict.get("magnitude") or 0.95
        self.saros = eclipse_dict.get("saros", "")

        # Greatest-eclipse coordinates
        self.ge_lat = parse_coord(self._raw.get("latitude", "0N"))
        self.ge_lon = parse_coord(self._raw.get("longitude", "0E"))

        # Path width (km) — 0 for partial eclipses
        pw = self._raw.get("path_width_km", "-")
        try:
            self.path_width_km = float(pw)
        except (ValueError, TypeError):
            self.path_width_km = 0.0

        # Gamma (shadow-axis distance from Earth center, in Earth radii)
        try:
            self.gamma = float(self._raw.get("gamma", "0"))
        except (ValueError, TypeError):
            self.gamma = 0.0

    def __repr__(self):
        return (f"Eclipse({self.date_raw}, {self.type_name}, "
                f"mag={self.magnitude})")

    # ---- properties ----

    @property
    def is_central(self):
        """True for Total, Annular, or Hybrid eclipses."""
        return self.path_width_km > 0

    @property
    def path_half_width(self):
        """Half the path width at greatest eclipse (km)."""
        return self.path_width_km / 2.0

    # ---- descriptive ----

    def describe(self):
        """Return a human-readable multi-line summary."""
        lines = [
            f"Date:       {self.date_raw}",
            f"Type:       {self.type_name}  (magnitude {self.magnitude})",
            f"GE point:   {self.ge_lat:.1f}°N, {self.ge_lon:.1f}°E",
            f"Gamma:      {self.gamma}",
            f"Saros:      {self.saros}",
        ]
        if self.is_central:
            lines.append(f"Path width: {self.path_width_km:.0f} km")
        return "\n".join(lines)

    # ---- path bearing estimation ----

    def estimate_path_bearing(self):
        """
        Estimate the eclipse-path bearing from the date.

        The Moon's shadow sweeps west → east.  Whether the path
        tilts NE or SE depends on the rate of change of the Sun's
        declination:
          • Spring (Sun moving north) → NE path  (~40°)
          • Fall   (Sun moving south) → SE path  (~140°)
          • Near solstice             → ~due east (~90°)
        """
        parts = self.date_raw.split()
        month = MONTH_NUM.get(parts[1], 6) if len(parts) > 1 else 6
        day = int(parts[2]) if len(parts) > 2 else 15
        doy = (month - 1) * 30.4 + day

        dec_rate = np.cos(2 * np.pi * (doy - 80) / 365.25)
        return 90.0 - dec_rate * 50.0

    # ---- effective path width ----

    def effective_half_width(self, dist_from_ge_km):
        """
        Path half-width adjusted for distance from greatest eclipse.
        The umbral shadow narrows as it moves away from the point of
        greatest eclipse because the shadow cone diverges and the
        Earth's curvature increases the slant distance.
        """
        if not self.is_central:
            return 0.0
        factor = max(0.5, 1.0 - dist_from_ge_km / 3000.0 * 0.6)
        return self.path_half_width * factor

    # ---- perpendicular distance to path ----

    def perpendicular_distance_km(self, obs_lat, obs_lon):
        """
        Estimate the observer's perpendicular distance (km) from the
        eclipse centerline using cross-track geometry.

        Steps:
        1. Gamma-based latitude gate  (high |γ| → narrow band)
        2. Bearing-constrained search (±2° around estimated bearing)
        3. Spherical cross-track formula
        """
        R = 6371.0
        d13_km = haversine_km(obs_lat, obs_lon, self.ge_lat, self.ge_lon)

        if d13_km < 200:
            return d13_km          # close enough — direct distance OK

        # Latitude gate
        ga = min(abs(self.gamma), 0.999)
        lat_range = np.degrees(np.arccos(ga)) * 0.55 + 5.0
        if abs(obs_lat - self.ge_lat) > lat_range:
            return d13_km

        d13 = d13_km / R
        lat1, lon1 = np.radians(self.ge_lat), np.radians(self.ge_lon)
        lat2, lon2 = np.radians(obs_lat), np.radians(obs_lon)
        dlon = lon2 - lon1
        x = np.sin(dlon) * np.cos(lat2)
        y = (np.cos(lat1) * np.sin(lat2)
             - np.sin(lat1) * np.cos(lat2) * np.cos(dlon))
        brg_obs = np.arctan2(x, y)

        bearing = self.estimate_path_bearing()
        min_xt = d13_km

        for b_off in range(-2, 3):
            brg_path = np.radians(bearing + b_off)
            xt = abs(R * np.arcsin(
                np.clip(np.sin(d13) * np.sin(brg_obs - brg_path), -1, 1)
            ))
            if xt < min_xt:
                cos_xt = np.cos(xt / R)
                if cos_xt > 1e-10:
                    cos_at = np.clip(np.cos(d13) / cos_xt, -1, 1)
                    at = R * np.arccos(cos_at)
                    if at <= 7500:
                        min_xt = xt
        return min_xt

    # ---- parallax-based viewer offset ----

    def parallax_offset(self, obs_lat, obs_lon):
        """
        Moon's apparent offset from the Sun's center for this observer,
        in Sun-radii units.

        Physics:
        The Moon is ~384 400 km away, so an observer displaced from
        the shadow centerline sees the Moon shifted by the topocentric
        parallax.  The known path width calibrates the conversion:

            At perpendicular distance = effective_half_width :
                offset = (magnitude − 1) × sun_radius

        which is *exactly* the threshold where the Moon just barely
        covers the Sun (edge of totality).

        Returns
        -------
        offset : float
            0       → Moon centered on Sun
            < mag−1 → totality (Moon fully covers Sun)
            > mag−1 → partial (some Sun visible)
            > 1+mag → no eclipse visible
        """
        sun_r = 1.0
        moon_r = self.magnitude * sun_r
        max_off = sun_r + moon_r

        dist_km = haversine_km(obs_lat, obs_lon, self.ge_lat, self.ge_lon)

        if self.is_central:
            eff_half = self.effective_half_width(dist_km)
            perp_km = self.perpendicular_distance_km(obs_lat, obs_lon)

            # Parallax-calibrated offset:
            #   offset / (mag − 1) = perp_km / eff_half
            # So at the path edge offset = (mag−1) exactly.
            if eff_half > 0:
                offset = perp_km * (self.magnitude - 1) * sun_r / eff_half
            else:
                offset = dist_km / 3500.0 * max_off

            return min(offset, max_off + 0.3)
        else:
            # Partial eclipse — gamma-based base offset
            base = abs(self.gamma) * 0.6
            return min(base + dist_km / 3500.0 * 0.6, max_off + 0.3)


# ---- backward-compatible wrapper used by draw_eclipse_sky ----
def viewer_offset(obs_lat, obs_lon, eclipse):
    """Wrapper: create Eclipse object and call parallax_offset."""
    return Eclipse(eclipse).parallax_offset(obs_lat, obs_lon)


# Keep standalone helpers available for the drawing function
def get_eclipse_params(eclipse):
    """Unpack useful numbers from an eclipse dict."""
    raw = eclipse.get("_raw", {})
    ecl_lat = parse_coord(raw.get("latitude", "0N"))
    ecl_lon = parse_coord(raw.get("longitude", "0E"))
    magnitude = eclipse.get("magnitude") or 0.95
    pw = raw.get("path_width_km", "-")
    try:
        path_w = float(pw)
    except (ValueError, TypeError):
        path_w = 0.0
    gamma_str = raw.get("gamma", "0")
    try:
        gamma = float(gamma_str)
    except (ValueError, TypeError):
        gamma = 0.0
    return ecl_lat, ecl_lon, magnitude, path_w, gamma


print("✓ Eclipse class & geometry helpers loaded.")

✓ Geometry helpers loaded.


In [22]:
# ============================================================
# DRAWING FUNCTION — renders the observer's sky view
# ============================================================

def draw_eclipse_sky(eclipse, obs_lat, obs_lon, time_frac):
    """
    Render what the eclipse looks like from (obs_lat, obs_lon).

    time_frac : float in [-1, 1]
        -1 = Moon approaching from the right
         0 = mid-eclipse (closest approach)
        +1 = Moon departing to the left
    """
    ecl_lat, ecl_lon, mag, path_w, _ = get_eclipse_params(eclipse)

    sun_r  = 1.0
    moon_r = mag * sun_r
    offset = viewer_offset(obs_lat, obs_lon, eclipse)

    # Moon travels horizontally across the Sun
    travel = sun_r + moon_r + 0.5
    mx = -time_frac * travel       # right → left
    my = offset

    # Center-to-center distance & obscuration
    d   = np.sqrt(mx**2 + my**2)
    obs_frac = overlap_fraction(sun_r, moon_r, d)

    dist_km = haversine_km(obs_lat, obs_lon, ecl_lat, ecl_lon)
    ecl_type = eclipse.get("type", "?")

    # ---- figure ----
    fig, ax = plt.subplots(figsize=(7, 7))
    ax.set_xlim(-2.5, 2.5)
    ax.set_ylim(-2.5, 2.5)
    ax.set_aspect("equal")

    # Sky color: darken with obscuration
    bright = max(0.02, 0.12 - 0.10 * obs_frac)
    ax.set_facecolor((bright, bright, bright + 0.12))

    # Corona glow — visible when >90 % obscured
    if obs_frac > 0.90:
        alpha_base = (obs_frac - 0.90) / 0.10 * 0.45
        for i in range(10):
            r = sun_r + 0.05 * (i + 1)
            a = alpha_base * (1 - i / 10)
            ax.add_patch(plt.Circle((0, 0), r,
                         color="#FFE4B5", alpha=max(a, 0), lw=0))

    # Sun disk
    ax.add_patch(plt.Circle((0, 0), sun_r, color="#FFD700", zorder=1))

    # Subtle limb-darkening rings
    for i in range(4):
        ax.add_patch(plt.Circle((0, 0), sun_r * (1 - 0.05 * i),
                     fill=False, ec="#FFA500", alpha=0.08, lw=1, zorder=1))

    # Moon disk
    ax.add_patch(plt.Circle((mx, my), moon_r, color="#111111", zorder=2))
    ax.add_patch(plt.Circle((mx, my), moon_r,
                 fill=False, ec="#333333", lw=1.5, zorder=3))

    # Stars when sky is dark enough
    if obs_frac > 0.85:
        rng = np.random.RandomState(42)
        n_stars = int(30 * (obs_frac - 0.85) / 0.15)
        sx = rng.uniform(-2.4, 2.4, n_stars)
        sy = rng.uniform(-2.4, 2.4, n_stars)
        ax.scatter(sx, sy, s=1, color="white", alpha=0.6, zorder=0)

    # Title & info
    ax.set_title(
        f"Eclipse View:  {eclipse['date_raw']}  —  {ecl_type}\n"
        f"Observer: {obs_lat:.1f}°N, {obs_lon:.1f}°E   |   "
        f"{dist_km:,.0f} km from center   |   "
        f"Obscuration: {obs_frac:.1%}",
        color="white", fontsize=11, fontweight="bold", pad=14,
    )
    info = f"Magnitude: {mag}   |   Saros: {eclipse.get('saros','?')}"
    if path_w > 0:
        info += f"   |   Path width: {path_w:.0f} km"
    ax.text(0, -2.35, info, ha="center", color="#aaaaaa", fontsize=9)
    ax.axis("off")
    plt.tight_layout()
    plt.show()

print("✓ Drawing function loaded.")

✓ Drawing function loaded.


In [23]:
# ============================================================
# INTERACTIVE VIEWER
# ============================================================
# • Dropdown — pick any of the 224 eclipses
# • Time slider — animate the Moon crossing the Sun
# • Lat / Lon sliders — move your observer position
#
# Tip: set continuous_update=False so the plot only redraws
#      when you release the slider (much smoother).
# ============================================================

# Build dropdown options
eclipse_options = {}
for i, e in enumerate(eclipse_list):
    label = f"{e['date_raw']}  —  {e['type']}  (Mag: {e['magnitude']})"
    eclipse_options[label] = i

def _update(idx, time_frac, lat, lon):
    draw_eclipse_sky(eclipse_list[idx], lat, lon, time_frac)

interact(
    _update,
    idx=Dropdown(options=eclipse_options, value=0, description="Eclipse:"),
    time_frac=FloatSlider(
        min=-1.0, max=1.0, step=0.02, value=-1.0,
        description="Time:", continuous_update=False,
        style={"description_width": "60px"},
        layout={"width": "500px"},
    ),
    lat=FloatSlider(
        min=-90, max=90, step=0.5, value=0.0,
        description="Lat (°N):", continuous_update=False,
        style={"description_width": "70px"},
        layout={"width": "500px"},
    ),
    lon=FloatSlider(
        min=-180, max=180, step=0.5, value=0.0,
        description="Lon (°E):", continuous_update=False,
        style={"description_width": "70px"},
        layout={"width": "500px"},
    ),
);

interactive(children=(Dropdown(description='Eclipse:', options={'2001 Jun 21  —  Total  (Mag: 1.0495)': 0, '20…